In [157]:

import numpy as np
import cvxpy as cvx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


def linear_kernel(x1, x2):
    
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))


class SVM(object):
    
    def __init__(self, kernel='linear', C=1.0):
        self.kernel = kernel
        self.C = C
    
        if self.C is not None:
            self.C = float(self.C)

    def _kernel_transform(self, X_train, kernel):
        if kernel == 'linear':
            kfunc = linear_kernel
        elif kernel == 'polynomial':
            kfunc = polynomial_kernel
        elif kernel == 'gaussian':
            kfunc = gaussian_kernel
            
        self.kfunc = kfunc
        
        n_samples, n_features = X_train.shape
        self.n_samples = n_samples 
        self.n_features = n_features 
    
        K = np.zeros((n_samples, n_samples))
        
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = kfunc(X_train[i], X_train[j])
             
        return K

    def __solve(self, K_matrix, y_train, C):
        
        n = y_train.shape[0] 
        y_= np.diag(y_train)  # 180 by 180(1 & -1) 
        alpha = cvx.Variable(n) 
        # constraints 
        constraints = []
        for i in range(n):
            constraints += [alpha[i] >= 0, alpha[i] <= C] 
        constraints += [y_train * alpha == 0]
        self.model_obj = cvx.Maximize(np.ones(n) * alpha - .5 * cvx.quad_form(alpha, y_.dot(K_matrix).dot(y_)))#convex식 
        #quad form 
        self.model = cvx.Problem(objective=self.model_obj, constraints=constraints)
        self.model.solve() 
        
        
        #support vector 
        self.a = np.array(alpha.value).flatten()
        self.sv = self.a > 1e-5 
        self.ind = np.arange(len(self.a))[self.sv] 
        self.a = self.a[self.sv] 
        self.sv_y = y_train[self.sv]
        self.sv_x= self.X_train[self.sv] 
        
        # intercept
        self.b = 0 
        for n in range(len(self.a)):
            self.b+=self.sv_y[n]
            self.b-=np.sum(self.a*self.sv_y*K_matrix[self.ind[n],self.sv])
        self.b/=len(self.a)
        
        #weight vector
        if self.kernel == 'linear':
            self.w = np.zeros(self.n_features)
            for n in range(len(self.a)):
                    self.w += self.a[n] * self.sv_y[n] * self.sv_x[n]
        else:
            self.w = None
            
    
    def predict(self,X):
        if self.w is not None:
           
            return np.dot(X,self.w) + self.b
          
        else: 
            y_predict = np.zeros(len(X))#
            
            for i in range(len(X)):
                s=0
                for a, sv_y, sv_x in zip(self.a, self.sv_y, self.sv_x):
                    s += a * sv_y * self.kfunc(X[i], sv_x)
                y_predict[i] = s
                
            return y_predict + self.b 
    def signpredict(self, X_test):
        
        return np.sign(self.predict(X_test))
    
    def fit(self, X_train, y_train):
        self.X_train = X_train 
        K = self._kernel_transform(X_train, self.kernel) 
        self.K = K
        self.__solve(K, y_train, self.C)


In [ ]:
def linear_kernel(x1, x2):
    
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))


In [ ]:
model= SVM(kernel = 'gaussian', C=100.00)

In [108]:
    def gen_non_lin_separable_data():
        mean1 = [-1,2]
        mean2 = [1,-1]
        mean3 = [4,-4]
        mean4 = [-4,4]
        cov = [[1.0,0.8],[0.8,1.0]]
        
        X1 = np.random.multivariate_normal(mean1, cov, 50) 
        X1 = np.vstack((X1, np.random.multivariate_normal(mean3,cov,50)))
        y1 = np.ones(len(X1)) 
        X2 = np.random.multivariate_normal(mean2, cov, 50)
        X2 = np.vstack((X2,np.random.multivariate_normal(mean4,cov,50)))
        y2 = np.ones(len(X2))*-1
        
        return X1, y1, X2, y2

In [109]:
    def split_train(X1,y1,X2,y2): 
        scaler = StandardScaler()
        
        X1_train = X1[:90]
        y1_train = y1[:90]
        X2_train = X2[:90]
        y2_train = y2[:90]
        X_train = np.vstack((X1_train, X2_train))
        y_train = np.hstack((y1_train, y2_train))
        
        X1_test = X1[90:]
        y1_test = y1[90:]
        X2_test = X2[90:]
        y2_test = y2[90:]
        X_test = np.vstack((X1_test,X2_test))
        y_test = np.hstack((y1_test,y2_test))
        
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        return X_train, y_train, X_test, y_test 

In [110]:
X1,y1,X2,y2 = g.gen_non_lin_separable_data()
X_train, y_train, X_test, y_test = g.split_train(X1,y1,X2,y2)
print(X_train, y_train, X_test, y_test)

In [164]:
import generate as g
import svm_yooyeon as s

X1,y1,X2,y2 = g.gen_non_lin_separable_data()#
X_train, y_train, X_test, y_test = g.split_train(X1,y1,X2,y2)
print(X_train, y_train, X_test, y_test)
print("X_train:",len(X_train),"X_test:",len(X_test),"Y_train:",len(y_train),"Y_test:",len(y_test))

[[ 0.23534505  0.87616773]
 [-0.34242724  0.36097239]
 [-0.32023191  0.54442753]
 [-0.3093839   0.74927531]
 [-0.31245583  0.56082735]
 [-0.84716011  0.01893477]
 [-0.13615001  0.68903742]
 [-0.18045124  0.66593834]
 [-0.54547879  0.21639478]
 [-0.67247054  0.42910484]
 [ 0.03673463  1.08998039]
 [-0.04975031  1.05922865]
 [-0.30514542  0.36249528]
 [-0.22265914  0.77150158]
 [-0.53665024  0.13924504]
 [-0.17036832  0.5771982 ]
 [-0.46758167  0.88400961]
 [-0.01983986  0.96199463]
 [-0.24199913  0.58032978]
 [-0.59708597  0.31607705]
 [-0.38450569  0.29914693]
 [-0.8342253   0.15176668]
 [ 0.1905929   0.73887961]
 [-0.52996028  0.18379794]
 [-0.24877023  0.5962069 ]
 [-0.85210792  0.29369629]
 [-0.13438383  0.69309589]
 [-0.2291018   0.59048931]
 [-0.18523259  0.55039692]
 [-0.10784627  1.15905992]
 [-0.72071785  0.29809001]
 [-0.03576105  0.51249252]
 [-0.29573104  0.52658448]
 [-0.13350692  0.51090278]
 [-1.29327605 -0.033384  ]
 [-0.10082902  0.92461397]
 [-1.17643574  0.1105841 ]
 

In [111]:
model= SVM(kernel = 'gaussian', C=100.00)

In [112]:
model.fit(X_train, y_train)

In [ ]:
def fit(self, X_train, y_train):
        self.X_train = X_train 
        K = self._kernel_transform(X_train, self.kernel) 
        self.K = K
        self.__solve(K, y_train, self.C)

In [ ]:
def _kernel_transform(self, X_train, kernel):
        #choose Kernel 
        if kernel == 'linear':
            kfunc = linear_kernel
        elif kernel == 'polynomial':
            kfunc = polynomial_kernel
        elif kernel == 'gaussian':
            kfunc = gaussian_kernel
            
        self.kfunc = kfunc
        
        # Kernel Matrix
        #Number of observations and features in model(input:X_train)
        n_samples, n_features = X_train.shape
        self.n_samples = n_samples 
        self.n_features = n_features 
        
        K = np.zeros((n_samples, n_samples))
        
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = kfunc(X_train[i], X_train[j]) #(180*180)
             
        return K

In [73]:
from IPython.display import HTML, display
data=model.K #(180*180)
display(HTML(
    '<table><tr>{}</tr></table>'.format(
        '</tr><tr>'.join(
            '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in table)
        )
 ))


1.66887285685,1.610228978,1.67334091431,0.883189783345,0.662219656069,1.96066167335,0.694520356056,0.382101703512,0.782092374842,1.60368273408,1.03971479076,2.14406608182,1.82449556309,0.971832890968,1.31946599261,0.892553759789,1.64732135807,0.750217263627,0.715597989454,1.60530279274,1.70286727667,0.585680751971,1.65554939418,0.795921053578,1.36795270403,1.58752726133,0.615436696184,0.994895814043,1.53229075283,0.943273024439,-0.105304419897,1.31716980273,1.57188914737,1.51370869397,0.639581557239,1.58733839768,1.18583167254,1.26898537641,1.1307274698,1.08502472783,0.664779770639,1.91624889849,0.256981772295,0.612999251286,1.59500618687,1.32831309566,1.3682462163,1.81107800623,2.31218346028,0.912801878425,0.559767409897,0.411930675095,1.43286440254,1.09266763671,1.17856361979,1.95271209446,1.48497522017,2.36370774961,1.31113427703,1.29805047138,0.640638058332,0.582202208982,1.19699596619,1.06551074787,2.40847856162,0.876888275303,1.60059167978,0.651526174631,1.57426517707,1.33525238471,0.228879478509,2.23513794404,2.03842367344,1.47315836054,1.88943382081,1.60572692189,1.74236732798,0.392093070141,1.95266675764,1.3532464136,2.65164527697,0.349778382856,1.64690635872,1.86836109102,1.65994558876,1.41325940516,1.54785570962,1.17251398294,1.30463338997,1.73766515546,-1.44848902454,-1.35652628659,-0.661920370931,-1.80647160593,-1.68597118072,-1.50895411735,-1.66718297943,-1.08399680695,-2.26067541756,-1.10242723173,-1.75272620898,-0.621768895247,-1.05567711802,-1.80833485429,-1.02788676154,-2.15455078917,-0.243759169415,-1.18089959863,-2.00263768087,-1.58239675395,-2.05498722062,-1.90239648466,-2.66322143404,-0.987342388533,-0.918063590828,-1.54205264009,-1.15800601932,-0.691859621423,-2.08990440428,-0.771491030675,-0.638915610213,-0.446545799555,-2.40474728933,-2.09418272572,-1.61860838091,-1.59603170356,-1.86476311143,-1.96582011964,-0.968284910076,-1.58256517896,-1.70177044288,-0.662874429875,-0.943062642601,-0.286258259797,-0.576837111417,-0.8614874198,-1.10418715179,-0.991747386173,-2.25292269785,-1.90502819562,-1.55055942664,-0.74042577759,-1.3497957383,-0.331760048453,-0.324104794678,-1.23789633298,-1.25153320082,-1.14341607251,-1.3721002316,-0.787598158377,-3.08339107266,-0.532168696254,-1.11798523433,-0.711086286108,-1.8701524571,-0.567155329418,-0.631991741894,-1.28164661322,-0.970025057381,-1.04797846981,-1.10503476887,-1.83854422162,-1.24988408127,-0.816091619544,-1.05635847194,-1.7401160254,-0.470745039108,-1.99821677793,-1.1617513604,-1.29267482238,-1.73078910299,-1.81510043555,-1.01118608042,-0.995682679756,-1.35883893257,-0.684847700009,-0.773082951807,-0.884697682998,-0.979889690157,-1.74336466759
1.610228978,1.82425004229,2.19131509961,1.26674422972,0.430890319469,1.97589296033,0.18629115259,-0.173857305203,0.228209861297,1.55885668271,1.03626371732,2.44372408298,1.86660293272,0.399857812916,1.33323314442,-0.25471066214,1.23763671726,0.545807034017,0.149278369176,1.3095995227,1.77368597775,-0.130031858656,1.7284633857,0.421506492765,1.37676314908,1.63619336679,-0.624318585719,0.288380757959,1.62011486717,0.457015391505,-1.44633603027,0.503089552445,2.01761026337,1.61796327447,0.10705660064,1.3930921937,0.77791381951,0.499496859349,0.555856509749,0.575647736025,0.238595637299,2.41301139488,-0.37720762705,-0.0764552141645,1.45494469295,1.25499723517,0.652405543991,1.72887925667,2.26152335226,0.0180156935719,0.482721741009,-0.252238538872,1.51093317698,1.2105101531,1.75370049809,2.74414583421,1.14489352197,2.80089523069,0.990805761435,0.845358894743,0.0669090511361,0.0296326326639,0.756718041038,0.277282759236,2.92163199915,0.750492317214,1.6449861665,-0.605710894005,1.33401351483,1.02444448815,-0.468203628766,2.3123752118,2.13940308299,1.2608243387,2.18343356477,1.59314465857,1.80288897952,-0.607083419779,1.68456963773,0.587930714914,3.48302684881,0.0879053324997,1.14387894083,1.53162693954,1.92015406742,1.76486512369,1.36816308633,0.840589040297,1.12622623492,1.68548703233,-1.63730058124,-0.993498231069,-0.684650

In [ ]:
# use cvxpy to solve dual equation (convex optimization problem)
# cvxpy modeling: features, constraints, optimal equation
#               : alpha, w, b, support vectors(x & y)
def __solve(self, K_matrix, y_train, C):
      
    n = y_train.shape[0]  
    y_= np.diag(y_train) # 180 by 180(composed of 1 & -1)
    
    alpha = cvx.Variable(n)  

In [ ]:
# alpha constraints 
    constraints = []
        for i in range(n):
            constraints += [alpha[i] >= 0, alpha[i] <= C] 
        constraints += [y_train * alpha == 0]

In [155]:
from IPython.display import Math
Math(r's.t. \sum_{i=1}^na_iy_i = 0 \\and\\ 0<=a_i<=C')

<IPython.core.display.Math object>

In [ ]:
# model's optimal equation(dual function) and result 
        self.model_obj = cvx.Maximize(np.ones(n) * alpha - .5 * cvx.quad_form(alpha, y_.dot(K_matrix).dot(y_))) 
        self.model = cvx.Problem(objective=self.model_obj, constraints=constraints)
        self.model.solve() 

In [151]:
from IPython.display import Math
Math(r'max L_D = \sum_{i=1}^na_i - \frac{1}{2}\sum_{i=1}^n\sum_{j=1}^na_ia_jy_iy_jK(x_i,x_j)')

<IPython.core.display.Math object>

In [ ]:
 # alphas, support vectors 
        # 2-dimension -> 1-dimension
        self.a = np.array(alpha.value).flatten()
        # constraint: a > 0
        self.sv = self.a > 1e-5  
        self.ind = np.arange(len(self.a))[self.sv] 
        self.a = self.a[self.sv] 
        self.sv_y = y_train[self.sv]
        self.sv_x= self.X_train[self.sv]  

In [124]:
# alphas
print('- alphas of the model:\n', model.a)

- Alphas of the model:
 [ 30.67474583  67.04244931  99.99999933  99.99999966  38.11494161
  99.99999977  89.98887345   2.25385489  99.99999954  99.99999995
  99.99999979  11.11141436  59.05428111  98.22005097  59.68912001]


In [158]:
# support vector's x values
print('- support vector x values of the model:\n',model.sv_x)
len(model.sv_x)

- support vector x values of the model:
 [[ 0.47089355  1.24642284]
 [-0.29107762  0.32581831]
 [-0.52134448  0.05990857]
 [-0.47978544  0.08773656]
 [-1.14885474  0.26873144]
 [ 0.11349822  0.72630892]
 [ 1.28090293 -1.03127924]
 [ 0.84583706 -1.58578819]
 [ 0.22976561 -0.16153275]
 [-0.13220388 -0.41185441]
 [ 0.58786124  0.16589611]
 [ 0.71592853  0.21609775]
 [ 0.1753085  -0.23112449]
 [-1.43251518  0.68665797]
 [-0.46578659  1.63957133]]


15

In [159]:
# support vector's y values
print('- support vector y values of the model:\n',model.sv_y)
len(model.sv_y)

- support vector y values of the model:
 [ 1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1.]


15

In [ ]:
# intercept of the model
        self.b = 0 
        for n in range(len(self.a)):
            self.b+=self.sv_y[n]
            self.b-=np.sum(self.a*self.sv_y*K_matrix[self.ind[n],self.sv])
        self.b/=len(self.a)

In [144]:
from IPython.display import Math
Math(r'b = y_i-\sum_{i=1}^n(a_iy_iK(x_i,x_j))')


<IPython.core.display.Math object>

In [156]:
# intercept of the model
print('- intercept of the model:\n',model.b)

- intercepts of the model:
 -0.954319717866


In [ ]:
# weight vector of the model
        if self.kernel == 'linear':
            self.w = np.zeros(self.n_features)
            for n in range(len(self.a)):
                    self.w += self.a[n] * self.sv_y[n] * self.sv_x[n]
        else:
            self.w = None

In [167]:
from IPython.display import Math
Math(r' w = \sum_{i=1}^n(a_iy_ix_i)')

<IPython.core.display.Math object>

In [125]:
# weight vector of the model 
print('- Weight vectors of the model:\n',model.w)

- Weights of the model:
 None


In [ ]:
#predicted y from model
def predict(self,X):  
        if self.w is not None:
                   # wxT+b
            return np.dot(X,self.w) + self.b 
          
        else: 
            y_predict = np.zeros(len(X))   
            
            for i in range(len(X)):
                s=0
                for a, sv_y, sv_x in zip(self.a, self.sv_y, self.sv_x):
                    s += a * sv_y * self.kfunc(X[i], sv_x)  
                y_predict[i] = s
                   # w*kernel(x)+b
            return y_predict + self.b  

In [143]:
from IPython.display import Math
Math(r' w = \sum_{i=1}^n(a_iy_iK(x_i,x_j))')

<IPython.core.display.Math object>

In [131]:
# predicted y values of X_test
print('- Predicted y values of X_test:\n',model.predict(X_test))#( 754 * 2)


- Predicted y values of X_test:
 [ 4.4581406   2.91256616  2.04083023  2.38156581  2.63746601  4.98712059
  1.39395896  3.24610909  3.81727459  2.02541259 -4.94432892 -4.10344383
 -3.9002355  -5.17518492 -3.49016431 -4.88104101 -4.26143758 -2.3984097
 -3.82748458 -2.0740838 ]


In [ ]:
def signpredict(self, X_test):
                # signs of y_test
        return np.sign(self.predict(X_test))
    

In [145]:
from IPython.display import Math
Math(r'f(x) = sign(y_i-\sum_{i=1}^n(a_iy_iK(x_i,x_j))')

<IPython.core.display.Math object>

In [134]:
# predicted signs of y values of X_test 
y_predict = model.signpredict(X_test) 
print('- Predicted signs of y values of X_test:\n',y_predict)

- Predicted signs of y values of X_test:
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.]


In [135]:
correct = np.sum(y_predict == y_test)
print("%d out of %d predictions correct" % (correct, len(y_predict)))

20 out of 20 predictions correct
